In [2]:
import streamlit as st
import pandas as pd
import math
from pathlib import Path
import psycopg2
from sqlalchemy import create_engine, text
import leafmap.foliumap as leafmap
import streamlit as st
import geopandas as gpd
from shapely import wkt
import folium
from streamlit_folium import folium_static

In [3]:
# !pip install geopandas

In [4]:
conn = st.connection("postgresql", type="sql")

# Perform query.
df = conn.query("SELECT * FROM public.core_vessel limit 100;", ttl="10m")
df

2024-08-20 22:27:24.253 
  command:

    streamlit run C:\Users\Sandesh\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-08-20 22:27:24.416 No runtime found, using MemoryCacheStorageManager
2024-08-20 22:27:24.416 No runtime found, using MemoryCacheStorageManager


,vessel_id,vessel_name,imo,mmsi,flag,dwt,callsign,vessel_type_id,owner_id,registry_id,...,depth,latestpositiontimestamp,vesseltypeandsize,registered_owner_id,technical_manager_id,third_party_operator_id,ism_manager_id,vessel_cat_id,length_m,breadth_m
0,170825,SCARLETT MANX,9801718,232012242,UNITED KINGDOM,63878.0,None,10,5698,55431,...,NaN,NaT,"Panamax, 60'-100'dwt",None,None,None,None,27,NaN,NaN
1,168595,TAIKOO BRILLIANCE,9738727,357772000,PANAMA,37786.0,None,10,28240,55424,...,NaN,NaT,"Handymax, 35'-50'dwt",None,None,None,None,24,NaN,NaN
2,160021,NORN,9869758,319261700,CAYMAN ISLANDS,665.0,None,43,41980,0,...,NaN,NaT,Yacht,None,None,None,None,341,89.99,NaN
3,26050,St. Ignatius of Loyola,8805157,548573200,PHILIPPINES,1459.0,DUH3109,211,46908,0,...,11.5,NaT,-5'dwt,None,None,None,None,293,104.00,16.20
4,49164,2GO Maligaya,9263150,548168700,PHILIPPINES,6445.0,DUE4337,180,46908,56462,...,18.6,NaT,10'-20'gt,None,None,None,None,274,195.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,31045,Alfredo Labadie,8320652,770576247,URUGUAY,3974.0,CXOG,137,44305,56782,...,7.0,NaT,Other Service,None,None,None,None,553,90.56,17.71
96,35002,Draga D7,7053161,770576248,URUGUAY,4572.0,CXFO,137,44305,0,...,8.4,NaT,Other Service,None,None,None,None,553,91.00,15.60
97,46591,General Artigas,8952998,770576002,URUGUAY,NaN,CXMG,31,44305,0,...,3.6,NaT,Miscellaneous,None,None,None,None,463,47.00,23.70
98,745140,Midia 17,8861278,264900084,ROMANIA,1500.0,None,233,53077,0,...,4.0,NaT,Miscellaneous,None,None,None,None,518,NaN,NaN


In [12]:
conn = st.connection("postgresql", type="sql")

# Perform query.
df = conn.query("select * from reporting.mview_geometry_port_term_berth;", ttl="10m")
df

2024-08-08 21:54:04.897 No runtime found, using MemoryCacheStorageManager


,id,name,short_name,type,areasize,country,location
0,34287,NORNE,Norne,Hydrocarbon Field,None,NORWAY,0103000020E6100000010000000600000078138B8CB236...
1,32542,NARVIK ANCHORAGE,Narvik Anchorage,Anchorage,None,NORWAY,0103000020E6100000010000001200000025A50B774F56...
2,33656,CALDER FIELD,Calder Field,Hydrocarbon Field,None,United Kingdom,0103000020E610000001000000050000000A0EBF127D6F...
3,4538,PUERTO DE PASTELILLO,CUPAS,Sub Port,Small,CUBA,0103000020E610000001000000050000001B3A7607724E...
4,6678,MASSAWA (MITSIWA),ERMSW,Port,Large,ERITREA,0103000020E610000001000000070000004388283FF2B7...
...,...,...,...,...,...,...,...
76298,81273,Gorojima Scrap Metal Wharf,None,Terminal,None,JAPAN,0103000020E6100000010000000B000000B4DC4ED6CC13...
76299,81274,Muryoji Terminal,None,Terminal,None,JAPAN,0103000020E6100000010000000B00000058396E029113...
76300,81275,Ohama Terminal,None,Terminal,None,JAPAN,0103000020E610000001000000050000001216699C8413...
76301,81276,Ono Terminal,None,Terminal,None,JAPAN,0103000020E610000001000000050000003FEB6C4D7713...


In [3]:
df = conn.query("select count(vessel_id) from public.core_vessel;", ttl="10m")
df

2024-08-08 21:13:38.786 No runtime found, using MemoryCacheStorageManager


,count
0,1025248


In [5]:
df['count'][0]

1025248

In [5]:
query = """SELECT jsonb_build_object('type', 'FeatureCollection', 'features', jsonb_agg(feature))
FROM (
    SELECT jsonb_build_object('type', 'Feature', 'geometry', ST_AsGeoJSON(location)::jsonb, 'properties', to_jsonb(row) - 'location') AS feature
    FROM (

SELECT cv.mmsi
	,cv.imo
	,cv.vessel_name
	,cp.port_name
	,cp.short_name
	,cpa.activity_time AS estimated_time
	,cp.location
FROM core_port_activity cpa
INNER JOIN core_port cp ON cpa.port_id = cp.port_id
INNER JOIN core_vessel cv ON cv.vessel_id = cpa.vessel_id
WHERE  cv.mmsi= 246642000
	-- AND cpa.activity_type_id = 36
        )row
    ) features"""

In [7]:
result = conn.query(query, ttl="10m")
result

2024-08-01 14:18:30.188 No runtime found, using MemoryCacheStorageManager


,jsonb_build_object
0,"{'type': 'FeatureCollection', 'features': [{'t..."


In [10]:
result['jsonb_build_object'][0]

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-58.451911, -51.897752]},
   'properties': {'imo': 9556612,
    'mmsi': 246642000,
    'port_name': 'Mount Pleasant',
    'short_name': 'FKMPN',
    'vessel_name': 'Dintelstroom',
    'estimated_time': '2023-10-20T14:10:02'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-58.451911, -51.897752]},
   'properties': {'imo': 9556612,
    'mmsi': 246642000,
    'port_name': 'Mount Pleasant',
    'short_name': 'FKMPN',
    'vessel_name': 'Dintelstroom',
    'estimated_time': '2023-10-20T10:20:31'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-58.451911, -51.897752]},
   'properties': {'imo': 9556612,
    'mmsi': 246642000,
    'port_name': 'Mount Pleasant',
    'short_name': 'FKMPN',
    'vessel_name': 'Dintelstroom',
    'estimated_time': '2023-09-03T00:04:46'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordina

In [13]:
info = conn.query("SELECT reporting.get_vessel_activity_by_port (4066);", ttl="10m")
info

2024-08-08 22:50:32.628 No runtime found, using MemoryCacheStorageManager


,get_vessel_activity_by_port
0,"{'Incoming': {'port_id': 4066, 'vessel_counts'..."


In [12]:


st.title("Marker Cluster")
with st.echo():        
    # cities = "https://raw.githubusercontent.com/giswqs/leafmap/master/examples/data/us_cities.csv"
    # regions = "https://raw.githubusercontent.com/giswqs/leafmap/master/examples/data/us_regions.geojson"
    
    # m.add_points_from_xy(
    #     cities,
    #     x="longitude",
    #     y="latitude",
    #     color_column="region",
    #     icon_names=["gear", "map", "leaf", "globe"],
    #     spin=True,
    #     add_legend=True,
    # )
    m = leafmap.Map(center=[40, -100], zoom=4)
    regions = result['jsonb_build_object'][0]
    m.add_geojson(regions, layer_name="US Regions")
    m.to_streamlit(height=700)

RuntimeError: generator didn't yield

In [5]:
conn = st.connection("postgresql", type="sql")

2024-08-19 09:29:17.832 
  command:

    streamlit run C:\Users\Sandesh\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [6]:

df = conn.query("SELECT * FROM reference.port_hierarchy;", ttl="10m")

2024-08-18 22:39:35.618 No runtime found, using MemoryCacheStorageManager


In [7]:
query_energy = conn.query("select * From reporting.get_port_energy_emission(3);", ttl="10m")
query_energy

2024-08-19 09:41:33.597 No runtime found, using MemoryCacheStorageManager


,port_id,vessel_id,min_port_arrival,max_port_departure,port_docking_time_hr,berth_docking_time_hr,port_dwelling_time_hr,cold_ironing_mw,propulsion_consumption_mw,cold_ironing_mwh,...,n20_cold_ironing_emission,ch4_cold_ironing_emission,nox_cold_ironing_emission,pm10_cold_ironing_emission,co2_propulsion_consumption_emission,so2_propulsion_consumption_emission,n20_propulsion_consumption_emission,ch4_propulsion_consumption_emission,nox_propulsion_consumption_emission,pm10_propulsion_consumption_emission
0,3,295283,2023-06-20 06:58:35,2023-06-21 03:03:29,20.08,15.71,4.37,1.0,0.10,15.71,...,0.471,86.405,230.937,24.193,17807.261,298.022,1.481,0.185,335.043,34.615
1,3,86758,2023-03-31 22:21:13,2023-04-01 16:50:54,18.49,18.23,0.26,1.5,0.20,27.35,...,0.820,150.398,401.972,42.111,14284.471,239.064,1.188,0.148,268.762,27.767
2,3,62461,2023-01-29 10:51:20,2023-01-31 06:57:53,44.11,42.78,1.33,1.0,0.10,42.78,...,1.283,235.290,628.866,65.881,38317.582,641.282,3.186,0.398,720.944,74.484
3,3,22185,2023-07-20 12:25:05,2023-07-21 06:35:08,18.17,17.85,0.32,0.0,0.00,0.00,...,0.000,0.000,0.000,0.000,17477.135,292.497,1.453,0.182,328.832,33.973
4,3,127722,2023-08-20 13:13:17,2023-08-24 18:15:59,101.05,20.28,80.77,1.0,0.10,20.28,...,0.608,111.540,298.116,31.231,95254.354,1594.174,7.921,0.990,1792.208,185.162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,3,341735,2023-09-26 01:23:26,2023-09-27 01:34:51,24.19,23.60,0.59,1.0,0.10,23.60,...,0.708,129.800,346.920,36.344,21000.727,351.467,1.746,0.218,395.128,40.823
6685,3,378313,2023-11-07 16:51:05,2023-11-09 08:20:30,39.49,39.49,0.00,0.0,0.00,0.00,...,0.000,0.000,0.000,0.000,37989.647,635.793,3.159,0.395,714.774,73.847
6686,3,380152,2023-01-29 23:51:00,2023-01-31 19:53:45,44.05,43.64,0.41,1.0,0.10,43.64,...,1.309,240.020,641.508,67.206,38173.924,638.878,3.175,0.397,718.241,74.205
6687,3,340776,2023-04-10 21:48:34,2023-04-11 01:56:04,4.13,3.84,0.29,1.0,0.10,3.84,...,0.115,21.120,56.448,5.914,3598.842,60.230,0.299,0.037,67.712,6.996


In [14]:
query_energy.columns

Index(['port_id', 'vessel_id', 'min_port_arrival', 'max_port_departure',
       'port_docking_time_hr', 'berth_docking_time_hr',
       'port_dwelling_time_hr', 'cold_ironing_mw', 'propulsion_consumption_mw',
       'cold_ironing_mwh', 'propulsion_consumption_mwh',
       'co2_cold_ironing_emission', 'so2_cold_ironing_emission',
       'n20_cold_ironing_emission', 'ch4_cold_ironing_emission',
       'nox_cold_ironing_emission', 'pm10_cold_ironing_emission',
       'co2_propulsion_consumption_emission',
       'so2_propulsion_consumption_emission',
       'n20_propulsion_consumption_emission',
       'ch4_propulsion_consumption_emission',
       'nox_propulsion_consumption_emission',
       'pm10_propulsion_consumption_emission'],
      dtype='object')

In [16]:
x = query_energy.describe()
x

,port_id,vessel_id,min_port_arrival,max_port_departure,port_docking_time_hr,berth_docking_time_hr,port_dwelling_time_hr,cold_ironing_mw,propulsion_consumption_mw,cold_ironing_mwh,...,n20_cold_ironing_emission,ch4_cold_ironing_emission,nox_cold_ironing_emission,pm10_cold_ironing_emission,co2_propulsion_consumption_emission,so2_propulsion_consumption_emission,n20_propulsion_consumption_emission,ch4_propulsion_consumption_emission,nox_propulsion_consumption_emission,pm10_propulsion_consumption_emission
count,6689.0,6689.000000,6689,6689,6689.000000,6689.000000,6689.000000,6689.000000,6689.000000,6689.000000,...,6689.000000,6689.000000,6689.000000,6689.000000,6.689000e+03,6689.000000,6689.000000,6689.000000,6689.000000,6689.000000
mean,3.0,102612.235611,2023-07-01 09:39:08.188191488,2023-07-03 05:26:24.137697024,43.787913,31.274971,12.512903,1.174256,0.160371,32.112385,...,0.963385,176.615337,472.044541,49.452286,3.901949e+04,653.028958,3.244859,0.405610,734.150572,75.848705
min,3.0,18145.000000,2023-01-01 01:45:09,2023-01-01 13:38:30,0.010000,0.010000,-18.330000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-7.291186e+04,-1220.250000,-6.063000,-0.758000,-1371.834000,-141.731000
25%,3.0,41567.000000,2023-04-07 02:26:17,2023-04-08 23:26:29,11.660000,9.570000,0.320000,0.500000,0.020000,5.900000,...,0.177000,32.450000,86.730000,9.086000,9.068453e+03,151.769000,0.754000,0.094000,170.623000,17.628000
50%,3.0,60349.000000,2023-07-02 16:45:07,2023-07-04 10:27:55,21.160000,16.210000,0.580000,1.000000,0.100000,15.720000,...,0.472000,86.460000,231.084000,24.209000,1.831140e+04,306.459000,1.523000,0.190000,344.528000,35.595000
75%,3.0,127663.000000,2023-09-25 04:17:10,2023-09-26 16:09:27,43.520000,34.020000,1.980000,1.500000,0.100000,32.200000,...,0.966000,177.073000,473.267000,49.580000,3.872435e+04,648.089000,3.220000,0.403000,728.597000,75.275000
max,3.0,443395.000000,2023-12-30 06:39:17,2023-12-30 17:36:37,4517.170000,4517.170000,3141.840000,9.000000,2.745000,4971.560000,...,149.147000,27343.580000,73081.932000,7656.202000,4.345514e+06,72726.374000,361.373000,45.172000,81760.707000,8447.101000
std,0.0,88162.244373,NaN,NaN,106.703757,78.714769,70.836956,1.053787,0.233255,93.772831,...,2.813186,515.750825,1378.461303,144.410228,1.011194e+05,1692.330438,8.409091,1.051143,1902.557826,196.562606


In [17]:
col = ['co2_cold_ironing_emission', 'so2_cold_ironing_emission',
       'n20_cold_ironing_emission', 'ch4_cold_ironing_emission',
       'nox_cold_ironing_emission', 'pm10_cold_ironing_emission',
       'co2_propulsion_consumption_emission',
       'so2_propulsion_consumption_emission',
       'n20_propulsion_consumption_emission',
       'ch4_propulsion_consumption_emission',
       'nox_propulsion_consumption_emission',
       'pm10_propulsion_consumption_emission']

In [21]:
x[col].mean()

co2_cold_ironing_emission               456078.705246
so2_cold_ironing_emission                 8550.136537
n20_cold_ironing_emission                  855.442321
ch4_cold_ironing_emission                 4377.616145
nox_cold_ironing_emission                10301.564856
pm10_cold_ironing_emission                1827.742439
co2_propulsion_consumption_emission     560691.751944
so2_propulsion_consumption_emission      10205.850049
n20_propulsion_consumption_emission        882.682619
ch4_propulsion_consumption_emission        841.944719
nox_propulsion_consumption_emission      11369.791175
pm10_propulsion_consumption_emission      1924.409914
dtype: float64

In [29]:
1e-9

1e-09

In [30]:
import pandas as pd

# Assuming query_energy is your DataFrame
# Calculate the necessary statistics
co2_cold_ironing_sum_g = query_energy['co2_cold_ironing_emission'].sum()
co2_cold_ironing_mean_g = query_energy['co2_cold_ironing_emission'].mean()

co2_propulsion_sum_g = query_energy['co2_propulsion_consumption_emission'].sum()
co2_propulsion_mean_g = query_energy['co2_propulsion_consumption_emission'].mean()

total_co2_sum_g = co2_cold_ironing_sum_g + co2_propulsion_sum_g
total_co2_mean_g = co2_cold_ironing_mean_g + co2_propulsion_mean_g

# Convert from grams (g) to megatonnes (Mt)
conversion_factor = 1e-9

co2_cold_ironing_sum_mt = co2_cold_ironing_sum_g * conversion_factor
co2_cold_ironing_mean_mt = co2_cold_ironing_mean_g * conversion_factor

co2_propulsion_sum_mt = co2_propulsion_sum_g * conversion_factor
co2_propulsion_mean_mt = co2_propulsion_mean_g * conversion_factor

total_co2_sum_mt = total_co2_sum_g * conversion_factor
total_co2_mean_mt = total_co2_mean_g * conversion_factor

# Combine all the values into a single row DataFrame
summary_df = pd.DataFrame({
    'CO2 Cold Ironing Emission Sum (Mt)': [co2_cold_ironing_sum_mt],
    'CO2 Cold Ironing Emission Mean (Mt)': [co2_cold_ironing_mean_mt],
    'CO2 Propulsion Consumption Emission Sum (Mt)': [co2_propulsion_sum_mt],
    'CO2 Propulsion Consumption Emission Mean (Mt)': [co2_propulsion_mean_mt],
    'Total CO2 Emission Sum (Mt)': [total_co2_sum_mt],
    'Total CO2 Emission Mean (Mt)': [total_co2_mean_mt]
})

# Display the resulting DataFrame
summary_df


,CO2 Cold Ironing Emission Sum (Mt),CO2 Cold Ironing Emission Mean (Mt),CO2 Propulsion Consumption Emission Sum (Mt),CO2 Propulsion Consumption Emission Mean (Mt),Total CO2 Emission Sum (Mt),Total CO2 Emission Mean (Mt)
0,0.151861,0.000023,0.261001,0.000039,0.412862,0.000062


In [12]:
# Create a line plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(query_energy['min_port_arrival'], query_energy['port_docking_time_hr'], marker='o', label='Date1')
plt.plot(query_energy['max_port_departure'], query_energy['port_docking_time_hr'], marker='o', label='Date2')
plt.title('Time Frame Plot')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True)
plt.legend()
plt.show()

In [13]:
query_energy

,port_id,vessel_id,min_port_arrival,max_port_departure,port_docking_time_hr,berth_docking_time_hr,port_dwelling_time_hr,cold_ironing_mw,propulsion_consumption_mw,cold_ironing_mwh,...,n20_cold_ironing_emission,ch4_cold_ironing_emission,nox_cold_ironing_emission,pm10_cold_ironing_emission,co2_propulsion_consumption_emission,so2_propulsion_consumption_emission,n20_propulsion_consumption_emission,ch4_propulsion_consumption_emission,nox_propulsion_consumption_emission,pm10_propulsion_consumption_emission
0,3,295283,2023-06-20 06:58:35,2023-06-21 03:03:29,20.08,15.71,4.37,1.0,0.10,15.71,...,0.471,86.405,230.937,24.193,17807.261,298.022,1.481,0.185,335.043,34.615
1,3,86758,2023-03-31 22:21:13,2023-04-01 16:50:54,18.49,18.23,0.26,1.5,0.20,27.35,...,0.820,150.398,401.972,42.111,14284.471,239.064,1.188,0.148,268.762,27.767
2,3,62461,2023-01-29 10:51:20,2023-01-31 06:57:53,44.11,42.78,1.33,1.0,0.10,42.78,...,1.283,235.290,628.866,65.881,38317.582,641.282,3.186,0.398,720.944,74.484
3,3,22185,2023-07-20 12:25:05,2023-07-21 06:35:08,18.17,17.85,0.32,0.0,0.00,0.00,...,0.000,0.000,0.000,0.000,17477.135,292.497,1.453,0.182,328.832,33.973
4,3,127722,2023-08-20 13:13:17,2023-08-24 18:15:59,101.05,20.28,80.77,1.0,0.10,20.28,...,0.608,111.540,298.116,31.231,95254.354,1594.174,7.921,0.990,1792.208,185.162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,3,341735,2023-09-26 01:23:26,2023-09-27 01:34:51,24.19,23.60,0.59,1.0,0.10,23.60,...,0.708,129.800,346.920,36.344,21000.727,351.467,1.746,0.218,395.128,40.823
6685,3,378313,2023-11-07 16:51:05,2023-11-09 08:20:30,39.49,39.49,0.00,0.0,0.00,0.00,...,0.000,0.000,0.000,0.000,37989.647,635.793,3.159,0.395,714.774,73.847
6686,3,380152,2023-01-29 23:51:00,2023-01-31 19:53:45,44.05,43.64,0.41,1.0,0.10,43.64,...,1.309,240.020,641.508,67.206,38173.924,638.878,3.175,0.397,718.241,74.205
6687,3,340776,2023-04-10 21:48:34,2023-04-11 01:56:04,4.13,3.84,0.29,1.0,0.10,3.84,...,0.115,21.120,56.448,5.914,3598.842,60.230,0.299,0.037,67.712,6.996


In [58]:
df_cold = conn.query(f"select * FROM public.ref_vessel_propulsion_consumption where  vessel_category = 'Cargo vessels';", ttl="10m")
df_cold

2024-08-19 14:49:12.641 No runtime found, using MemoryCacheStorageManager


,min_dwt,max_dwt,vessel_category,propulsion_consumption
0,0,14000,Cargo vessels,0.200
1,14001,30000,Cargo vessels,0.300
2,30001,50000,Cargo vessels,0.400
3,50001,82000,Cargo vessels,0.490
4,82001,135000,Cargo vessels,0.490
5,135001,160000,Cargo vessels,0.625
6,160001,200000,Cargo vessels,0.760
7,200001,320000,Cargo vessels,1.125
8,320001,999999999,Cargo vessels,1.125


In [54]:
df_cold['min_gt'] = pd.to_numeric(df_cold['min_gt'], errors='coerce')
df_cold['max_gt'] = pd.to_numeric(df_cold['max_gt'], errors='coerce')

In [55]:
max_gt = 4999
min_gt = 150

In [57]:
average_hoteling_kw = int(df_cold[(df_cold['min_gt'] >= min_gt) & (df_cold['max_gt'] <= max_gt)]['average_hoteling_kw'].iloc[0])
type(average_hoteling_kw)

pandas.core.series.Series

In [75]:
df_prop = conn.query(f"select * FROM public.ref_vessel_propulsion_consumption where  vessel_category = 'Cargo vessels';", ttl="10m")
df_prop['min_dwt'] = pd.to_numeric(df_prop['min_dwt'], errors='coerce')
df_prop['max_dwt'] = pd.to_numeric(df_prop['max_dwt'], errors='coerce')
df_prop

2024-08-19 15:00:26.286 No runtime found, using MemoryCacheStorageManager


,min_dwt,max_dwt,vessel_category,propulsion_consumption
0,0,14000,Cargo vessels,0.200
1,14001,30000,Cargo vessels,0.300
2,30001,50000,Cargo vessels,0.400
3,50001,82000,Cargo vessels,0.490
4,82001,135000,Cargo vessels,0.490
5,135001,160000,Cargo vessels,0.625
6,160001,200000,Cargo vessels,0.760
7,200001,320000,Cargo vessels,1.125
8,320001,999999999,Cargo vessels,1.125


In [78]:
max_gt = int(30000)
min_gt = int(14001)
df_prop[(df_prop['max_dwt'] <= max_gt) & (df_prop['min_dwt'] >= min_gt)]

,min_dwt,max_dwt,vessel_category,propulsion_consumption
1,14001,30000,Cargo vessels,0.3


In [79]:

propulsion_consumption = df_prop[(df_prop['max_dwt'] <= max_gt) & (df_prop['min_dwt'] >= min_gt)]['propulsion_consumption'].iloc[0]
print(str(propulsion_consumption))

0.3


In [88]:
df = conn.query(f"SELECT * FROM public.ref_future_power_consumption where year between 2024 and 2025 and change_type in('Low','Medium','High','Cargo');", ttl="10m")
df

2024-08-20 14:25:13.338 No runtime found, using MemoryCacheStorageManager


,future_id,type,year,change_type,change_in_percentage
0,3893,Cold Ironing,2024,Low,5.0
1,3894,Cold Ironing,2025,Low,17.0
2,3987,Cold Ironing,2024,High,5.0
3,3988,Cold Ironing,2025,High,24.0
4,4034,Propulsion Adoption,2024,Low,0.7
5,4035,Propulsion Adoption,2025,Low,1.7
6,4128,Propulsion Adoption,2024,High,0.7
7,4129,Propulsion Adoption,2025,High,3.0
8,4175,Propulsion Distance,2024,Low,0.0
9,4176,Propulsion Distance,2025,Low,0.0


In [86]:
df[df['type'] == 'Traffic Forecast']

,future_id,type,year,change_type,change_in_percentage
18,4880,Traffic Forecast,2024,Cargo,0.0
19,4881,Traffic Forecast,2025,Cargo,-0.3


In [87]:
pivot_df = df[df['type'] == 'Cold Ironing'].pivot(index='year', columns='change_type', values='change_in_percentage')

pivot_df

change_type,High,Low,Medium
year,,,
2024,5.0,5.0,5.0
2025,24.0,17.0,20.0


In [89]:
pivot_df = df[df['type'] == 'Traffic Forecast'].pivot(index='year', columns='change_type', values='change_in_percentage')
pivot_df

change_type,Cargo
year,
2024,0.0
2025,-0.3


In [10]:
vessel_options = 'Cargo vessels'

In [11]:
df = conn.query(f"select distinct main_vessel_category FROM reference.ref_vessel_type_category where vessel_category = '{vessel_options}';", ttl="10m")
main_vessel_category= df['main_vessel_category'].iloc[0]
main_vessel_category

2024-08-20 22:28:51.517 No runtime found, using MemoryCacheStorageManager


'Cargo'

In [12]:
start_year = 2024
end_year = 2030

In [13]:
df = conn.query(f"SELECT * FROM public.ref_future_power_consumption where year between {start_year} and {end_year} and change_type in('Low','Medium','High','{main_vessel_category}');", ttl="10m")
df['year'] = pd.to_datetime(df['year'], format='%Y')

2024-08-20 22:28:55.089 No runtime found, using MemoryCacheStorageManager


In [14]:
df

,future_id,type,year,change_type,change_in_percentage
0,3893,Cold Ironing,2024-01-01,Low,5.0
1,3894,Cold Ironing,2025-01-01,Low,17.0
2,3895,Cold Ironing,2026-01-01,Low,29.0
3,3896,Cold Ironing,2027-01-01,Low,41.0
4,3897,Cold Ironing,2028-01-01,Low,53.0
...,...,...,...,...,...
65,4882,Traffic Forecast,2026-01-01,Cargo,-0.4
66,4883,Traffic Forecast,2027-01-01,Cargo,-0.6
67,4884,Traffic Forecast,2028-01-01,Cargo,-0.7
68,4885,Traffic Forecast,2029-01-01,Cargo,-0.9


In [17]:
df[df['type'] == 'Traffic Forecast']['change_in_percentage']

63    0.0
64   -0.3
65   -0.4
66   -0.6
67   -0.7
68   -0.9
69   -1.0
Name: change_in_percentage, dtype: float64

In [16]:
styled_df =df

In [ ]:
styled_df['new_cold_ironing_mw_vessel'] = styled_df['average_hoteling_MW'] * 

In [ ]:
pivot_df = df[df['type'] == 'Traffic Forecast'].pivot( columns='change_type', values='change_in_percentage')
